## CMG_CCS_4D_Seiscmic

Author: Hyunmin Kim | Last Update Date: 2024-02-19

In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, shutil
from tqdm import tqdm 
from multiprocessing import Pool
import pyvista as pv
from lib.Sim_CMG_CCS import Sim_CCS

In [ ]:
Current_dir = os.getcwd()
N_ensemble = 500
N_thread = 5

In [ ]:
NX, NY, NZ = 32, 32, 16
# instantiate the simulation object
sim_ccs = Sim_CCS()

data = np.load('ensemble_500.npz')
Facies = data['Facies']
Porosity = data['Porosity']
Permeability = data['Permeability']

# set the simulation parameters
input_parameters = {'porosity': Porosity,
                    'permeability': Permeability,
                    'Facies': Facies}

# set the bottom hole pressure constraint
maximum_bhp = 3500 # psi
sim_ccs.reset_bhp_constraint(maximum_bhp)

# set the injection amount
injection_rate = 15_839_619 # scf/day
sim_ccs.reset_injection_amout(injection_rate)

# set your CMG exe path
cmg_exe = '"C:\\Program Files\\CMG\\GEM\\2020.10\\Win_x64\\EXE\\gm202010.exe"'
sim_ccs.reset_parameters_to_play(list(input_parameters.keys()))
sim_ccs.reset_CMG_exe(cmg_exe)
# sim_ccs.remove_out_files()

In [ ]:
sim_ccs.ensemble_simulation(input_parameters=input_parameters)

In [ ]:
sim_ccs.N_ensemble = 500
sim_ccs.run_multiple_CMG(N_thread=5)

In [ ]:
f = open(os.path.join(run_dir, 'porosity.inc'))
lines = []
for line in f:
    if line == '*POR *ALL\n':
        continue
    lines.append(line.split(' ')[0])
por = np.array(lines, dtype=float).reshape((NZ, NY, NX))

In [ ]:
f = open(os.path.join(run_dir, 'permeability.inc'))
lines = []
for line in f:
    if line == '*PERMI *ALL\n':
        continue
    lines.append(line.split(' ')[0])
perm = np.array(lines, dtype=float).reshape((NZ, NY, NX))

In [ ]:
sim_ccs.CMG_GEM_data_file = 'cmg_ccs_run_file_v2.dat'

In [ ]:
sim_ccs.reset_Ptime([2025, 2026, 2027, 2028, 2029])
Sg =  sim_ccs.read_grid_results(run_dir='Test')['GasSaturation']

psi_to_bar = 0.0689476
P = sim_ccs.read_grid_results(run_dir='Test')['Pressure(psia)'] * psi_to_bar

F_to_K = lambda x: (x - 32) * 5 / 9 + 273.15

T = F_to_K(sim_ccs.read_grid_results(run_dir='Test')['Temperature(degF)'])
S = np.ones_like(P) * 0.02

ft_to_m = 0.3048
TOP_D = sim_ccs.read_top_depth(run_dir=run_dir) * ft_to_m
DZ = np.ones_like(por_map) * 6.25 * ft_to_m

## Calculate Vp in the reservoir

In [ ]:
from lib.seismogram import CCS_Seismogram

In [ ]:
ccs_seis = CCS_Seismogram(P=P, T=T, S=S,
                          por=por, Sg=Sg, TOP_D=TOP_D,
                          current_directory=os.getcwd(),
                          heat_capacity_file='Heat_capacity_of_co2.xlsx')

In [ ]:
Vp = ccs_seis.velocity_of_saturated_rock()
rho_sat = ccs_seis.rho_sat

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(2, 2, 1)
plt.imshow(Vp[1, :, 10, :] - Vp[0, :, 10, :])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Vp changes at 1yr (km/s)', fontdict={'fontsize': 25})

plt.subplot(2, 2, 2)
plt.imshow(Vp[2, :, 10, :] - Vp[0, :, 10, :])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Vp changes at 2yr (km/s)', fontdict={'fontsize': 25})

plt.subplot(2, 2, 3)
plt.imshow(Vp[3, :, 10, :] - Vp[0, :, 10, :], vmax=0)
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Vp changes at 3yr (km/s)', fontdict={'fontsize': 25})

plt.subplot(2, 2, 4)
plt.imshow(Vp[4, :, 10, :] - Vp[0, :, 10, :])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Vp changes at 4yr (km/s)', fontdict={'fontsize': 25})

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(2, 3, 1)
plt.imshow(Vp[3, :, 10, :])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Seismic velocity (km/s)', fontdict={'fontsize': 25})

plt.subplot(2, 3, 2)
plt.imshow(rho_sat[3, :, 10, :])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('density of rock (g/cm3)', fontdict={'fontsize': 25})

plt.subplot(2, 3, 3)
plt.imshow(P[3, :, 10, :])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Pressure (bar)', fontdict={'fontsize': 25})

plt.subplot(2, 3, 4)
plt.imshow(Sg[3, :, 10, :])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Gas saturation', fontdict={'fontsize': 25})

plt.subplot(2, 3, 5)
plt.imshow(por[:, 10, :])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Porosity', fontdict={'fontsize': 25})

plt.subplot(2, 3, 6)
plt.imshow(perm[:, 10, :])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Permeability', fontdict={'fontsize': 25})
plt.tight_layout()

## Poststacked seismic data extraction

In [ ]:
seismic_tdom, seismic = ccs_seis.seismic(DZ, TOP_D)

In [ ]:
flag = seismic_tdom[-1].max(axis = -1).max(axis = -1) > .001
seismic_init= seismic_tdom[0, flag]
seismic_5yr = seismic_tdom[-1, flag]
visual_grid = seismic_init - seismic_5yr
grid = pv.ImageData()
grid.dimensions = np.array(visual_grid[::-1].T.shape) + 1
grid.origin = (1, 1, 1)  # The bottom left corner of the data set
grid.spacing = (1, 1, 1/3)  # These are the cell sizes along each axis
grid.cell_data["values"] = visual_grid[::-1].T.flatten(order="F")  # Flatten the array
slice = grid.slice_along_axis(n=7, axis="y")#slice_orthogonal(x=9, y=9, z=10)
slice.plot(show_edges=False, cmap = 'seismic')

In [ ]:
plt.figure(figsize=(20,10))

AI = ccs_seis.Vp * ccs_seis.rho_sat * 1000 # Pa*s/m3
plt.subplot(2, 2, 1)
plt.imshow(AI[3, :, 10, : ])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Acoustic impedance', fontdict={'fontsize': 20})

plt.subplot(2, 2, 2)
plt.imshow(seismic[3, :, 10, :], cmap = 'seismic')
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Poststack seismic', fontdict={'fontsize': 20})

TWT = ccs_seis.two_way_travel_time(DZ, TOP_D)
plt.subplot(2, 2, 3)
plt.imshow(TWT[3, :, 10, : ])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('two-way travel time (sec)', fontdict={'fontsize': 20})

plt.subplot(2, 2, 4)
plt.imshow(Sg[3, :, 10, : ])
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('Gas saturation', fontdict={'fontsize': 20})


In [ ]:
time_lapse_seismic = np.ones_like(seismic)
for i in range(6):
    time_lapse_seismic[i] = seismic[i] - seismic[0]

plt.figure(figsize=(20,10))

plt.subplot(2, 2, 1)
plt.imshow(time_lapse_seismic[0, :, 10, : ], cmap = 'seismic', vmin = -1, vmax = 1)
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('time_lapse_seismic at 0yr', fontdict={'fontsize': 20})

plt.subplot(2, 2, 2)
plt.imshow(time_lapse_seismic[1, :, 10, :], cmap = 'seismic', vmin = -1, vmax = 1)
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('time_lapse_seismic at 1yr', fontdict={'fontsize': 20})

plt.subplot(2, 2, 3)
plt.imshow(time_lapse_seismic[2, :, 10, : ], cmap = 'seismic', vmin = -1, vmax = 1)
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('time_lapse_seismic at 2yr', fontdict={'fontsize': 20})

plt.subplot(2, 2, 4)
plt.imshow(time_lapse_seismic[3, :, 10, : ], cmap = 'seismic', vmin = -1, vmax = 1)
plt.colorbar()
plt.xlabel('X-coordinate')
plt.ylabel('Z-coordinate')
plt.title('time_lapse_seismic at 3yr', fontdict={'fontsize': 20})